# Modelo de Markowitz
Consideramos un problema de cartera clásico con n activos o acciones mantenidos a lo largo de un período de tiempo. Permitimos que $x_i$ denote la cantidad de activo $i$ mantenida durante todo el período, con $x_i$ en dólares, al precio al inicio del período. Una posición larga normal en el activo $i$ corresponde a $x_i > 0$; una posición corta en el activo $i$ (es decir, la obligación de comprar el activo al final del período) corresponde a $x_i < 0$. Permitimos que $p_i$ denote el cambio relativo en el precio del activo $i$ durante el período, es decir, su cambio de precio durante el período dividido por su precio al inicio del período. El rendimiento total de la cartera es $r = p^T x$ (expresado en dólares). La variable de optimización es el vector de la cartera $x \in \mathbb{R}^n$.
Se pueden considerar una amplia variedad de restricciones en la cartera. El conjunto más simple de restricciones es que $xi \geq 0$ (es decir, no se permiten posiciones cortas) y $1^T x = B$ (es decir, el presupuesto total a invertir es B, que a menudo se toma como uno).

Tomamos un modelo estocástico para los cambios de precio: $p \in \mathbb{R}^n$ es un vector aleatorio, con una media conocida $\bar{p}$ y una covarianza $\Sigma$. Por lo tanto, con una cartera $x \in \mathbb{R}^n$, el rendimiento $r$ es una variable aleatoria (escalar) con una media de $\bar{p}^Tx$ x y una varianza $x^T\Sigma x$. La elección de la cartera $x$ implica un equilibrio entre la media del rendimiento y su varianza.

Así, el modelo de optimización cuadrática de Markowitz es
$$\begin{matrix}
\text{minimizar} & x^T\Sigma x \\
\text{sujeto a}  & \bar{p}^Tx \geq r_\text{min} \\
\text{} & 1^Tx=1, & x \succ 0
\end{matrix}$$

In [1]:
import numpy as np
import datetime as dt
import yfinance as yf
from scipy import optimize as opt

En particular, se definen las variables $m \in \mathbb{R}^3, \Sigma \in \mathbb{R}^{3\times3}$

In [2]:
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

stockList = ["CBA.AX", "BHP.AX", "TLS.AX"]
stocks = [stock for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)
meanReturns, covMatrix = getData(stocks=stocks, start=startDate, end=endDate)
print(meanReturns, covMatrix)

[*********************100%%**********************]  3 of 3 completed
BHP.AX    0.000671
CBA.AX    0.000451
TLS.AX    0.000046
dtype: float64           BHP.AX    CBA.AX    TLS.AX
BHP.AX  0.000255  0.000040 -0.000002
CBA.AX  0.000040  0.000118  0.000026
TLS.AX -0.000002  0.000026  0.000071


In [3]:
def portfolioPerformance(weights: np.array, meanReturns, covMatrix: np.array):
    factor = np.sqrt(252)
    returns = np.sum(meanReturns*weights)*factor**2
    std = np.sqrt(weights.T @ covMatrix @ weights)*factor
    return returns, std

weights = np.array([0.3, 0.3, 0.4])
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)
print(returns, std) 

0.08945406107035113 0.12060890628561223


Se define el problema de optimización
$$\begin{matrix}
\text{minimizar} & -\frac{m^Tw - r_0}{w^T\Sigma w} \\
\text{sujeto a}  & 1^Tw=1, & w \succeq 0
\end{matrix}$$

Donde $f_0(w) = \frac{m^Tw - r_0}{w^T\Sigma w}$ es concavo y $-f_0$ es convexo. Se intuye que $\Sigma \succeq 0$.

In [4]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate=0):
    """
    Calcula la proporción negativa del índice de Sharpe para una cartera de activos dados.
    """
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate) / pStd

def maxSR(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0, 1)):
    """
    Encuentra la cartera de activos que maximiza el índice de Sharpe.

    Retorna:
    scipy.optimize.OptimizeResult: El resultado de la optimización que contiene los pesos óptimos.
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Restricción de suma de pesos igual a 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = opt.minimize(fun=negativeSR, x0=numAssets * [1. / numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

result = maxSR(meanReturns, covMatrix)
print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.8458203031697065
       x: [ 4.096e-01  5.904e-01  1.897e-17]
     nit: 4
     jac: [-6.548e-05  4.540e-05  4.541e-02]
    nfev: 16
    njev: 4


In [5]:
def portfolioVariance(weights, meanReturns, covMatrix):
    """
    Calcula la varianza de una cartera de activos dados.

    Retorna:
    float: La varianza de la cartera.
    """
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0, 1)):
    """
    Encuentra la cartera de activos que minimiza la varianza.

    Retorna:
    scipy.optimize.OptimizeResult: El resultado de la optimización que contiene los pesos óptimos.
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Restricción de suma de pesos igual a 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = opt.minimize(portfolioVariance, numAssets * [1. / numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

result = minimizeVariance(meanReturns, covMatrix)
print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.11096650978387466
       x: [ 1.615e-01  2.266e-01  6.119e-01]
     nit: 8
     jac: [ 1.111e-01  1.109e-01  1.110e-01]
    nfev: 32
    njev: 8


$$
\begin{matrix}
\text{minimizar} & w^T\Sigma w \\
\text{sujeto a}  & m^Tw - R_0 \\
\text{} & 1^Tw=1, & w \succeq 0
\end{matrix}
$$

In [6]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = opt.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

A partir de los problemas de optimización planteados, se obtiene información significativa

In [7]:
import pandas as pd

def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """
    Calcula resultados clave de una cartera de activos, incluyendo el portafolio con el mayor índice de Sharpe,
    el portafolio con la menor volatilidad, y la frontera eficiente.

    Parámetros:
    - meanReturns (pandas.Series): Tasas de retorno esperadas de los activos.
    - covMatrix (pandas.DataFrame): Matriz de covarianza de los activos.
    - riskFreeRate (float, opcional): Tasa libre de riesgo, por defecto es 0.
    - constraintSet (tuple, opcional): Conjunto de restricciones para los pesos de la cartera.

    Retorna:
    tuple: Una tupla que contiene los siguientes elementos:
        - maxSR_returns (float): Retorno del portafolio con el mayor índice de Sharpe.
        - maxSR_std (float): Desviación estándar (volatilidad) del portafolio con el mayor índice de Sharpe.
        - maxSR_allocation (pandas.DataFrame): Asignación de activos en el portafolio con el mayor índice de Sharpe.
        - minVol_returns (float): Retorno del portafolio con la menor volatilidad.
        - minVol_std (float): Desviación estándar (volatilidad) del portafolio con la menor volatilidad.
        - minVol_allocation (pandas.DataFrame): Asignación de activos en el portafolio con la menor volatilidad.
        - efficientList (list): Lista de valores de volatilidad para diferentes niveles de retorno en la frontera eficiente.
    """
     
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns * 100, 2), round(maxSR_std * 100, 2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i * 100, 0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns * 100, 2), round(minVol_std * 100, 2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i * 100, 0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList


In [17]:
maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList = calculatedResults(meanReturns, covMatrix)

In [18]:
print("Resultados del Portafolio con el Mayor Índice de Sharpe:")
print("Retorno esperado:", maxSR_returns, "%")
print("Volatilidad (Desviación estándar):", maxSR_std, "%")
print("Asignación de activos en el Portafolio con Mayor Sharpe:")
print(maxSR_allocation)

Resultados del Portafolio con el Mayor Índice de Sharpe:
Retorno esperado: 13.64 %
Volatilidad (Desviación estándar): 16.12 %
Asignación de activos en el Portafolio con Mayor Sharpe:
        allocation
BHP.AX        41.0
CBA.AX        59.0
TLS.AX         0.0


In [19]:
print("\nResultados del Portafolio con la Menor Volatilidad:")
print("Retorno esperado:", minVol_returns, "%")
print("Volatilidad (Desviación estándar):", minVol_std, "%")
print("Asignación de activos en el Portafolio con Menor Volatilidad:")
print(minVol_allocation)



Resultados del Portafolio con la Menor Volatilidad:
Retorno esperado: 6.01 %
Volatilidad (Desviación estándar): 11.1 %
Asignación de activos en el Portafolio con Menor Volatilidad:
        allocation
BHP.AX        16.0
CBA.AX        23.0
TLS.AX        61.0


In [20]:
print("\nResultados de la Frontera Eficiente:")
print("Niveles de Retorno Objetivo:")
print(np.linspace(minVol_returns, maxSR_returns, 20))
print("Volatilidad correspondiente a los Niveles de Retorno Objetivo:")
print(efficientList)


Resultados de la Frontera Eficiente:
Niveles de Retorno Objetivo:
[ 6.01        6.41157895  6.81315789  7.21473684  7.61631579  8.01789474
  8.41947368  8.82105263  9.22263158  9.62421053 10.02578947 10.42736842
 10.82894737 11.23052632 11.63210526 12.03368421 12.43526316 12.83684211
 13.23842105 13.64      ]
Volatilidad correspondiente a los Niveles de Retorno Objetivo:
[0.25357983726961897, 0.25357983726957367, 0.25357983726972894, 0.25357983727040634, 0.2535798372696585, 0.2535800811992366, 0.2535798372695801, 0.2535798372700303, 0.253579839239382, 0.25357983726954664, 0.25357983727097433, 0.2535798372700308, 0.25357983727018707, 0.25357983726964073, 0.25357983726953814, 0.25357983726965544, 0.253579837269671, 0.25357983726954136, 0.25357983727048306, 0.2535798373537825]
